# Installing Mindcontrol

1. Install meteor

    ```bash
    curl https://install.meteor.com/ | sh
    ```

2. Clone mindcontrol

    ```bash
    git clone https://github.com/akeshavan/mindcontrol.git
    ```

3. Start the server

    ```bash
    cd mindcontrol
    meteor
    ```

4. Navigate to http://localhost:3000 in your chrome browser.
You should see the default entries loaded into the mindcontrol database. It should look like:

    ![](images/mindcontrol_install_init.png)




Here we see the "Exams" and "Freesurfer" modules. We are going to add a new module called "Test". First, we need to prepare our data

# Preparing and Serving Data

### Data organization

Make sure your data has a consistent organization structure.  Your base directory should look like:
    
    /base/directory
             |___ sub-01
                   |___ anat
                         |___ sub-01_T1w.nii.gz
             |___ sub-02
                   |___ anat
                         |___ sub-01_T1w.nii.gz
             |___ sub-03
                   |___ anat
                         |___ sub-01_T1w.nii.gz
    

This is the BIDS data organization, and you can find example at https://github.com/INCF/BIDS-examples. Pay close attention to you **`/base/directory`** -- this is where we will start a static server

### Starting a static server

In your **`/base/directory`**, create a file called **`start_static_server.py`**. Copy/paste the following into the file:

```python
#!/usr/bin/env python
import http.server
import os

class MyHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header("Access-Control-Allow-Origin", "*")
        http.server.SimpleHTTPRequestHandler.end_headers(self)

if __name__ == '__main__':
        http.server.test(HandlerClass=MyHTTPRequestHandler, port=3002)
```

`cd` to **`/base/directory`** and run the static server:

```bash
python start_static_server.py
```

To test that it works, in your browser navigate to http://localhost:3002. You should see your directory structure

# Adding a New Module to Mindcontrol
Modules are groupings of similar data that can be used to filter and organize subjects for editing. For example, all data that has been processed by FreeSurfer will appear in the FreeSurfer module. You can find all modules at the top left of the Mindcontrol home window ::

![](images/mindcontrol_install_init_annot.png)

Each module is defined in the ./mindcontrol/private/generator.json